## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-21-16-54-24 +0000,bbc,Appeals court throws out Trump's $500m civil f...,https://www.bbc.com/news/articles/c5y09q1zgg8o...
1,2025-08-21-16-52-05 +0000,nyt,U.S. Navy Sailor Is Convicted of Spying for China,https://www.nytimes.com/2025/08/20/us/us-navy-...
2,2025-08-21-16-51-17 +0000,wapo,Uganda says it reached deal with U.S. to take ...,https://www.washingtonpost.com/world/2025/08/2...
3,2025-08-21-16-48-47 +0000,nyt,Walmart Recalls Frozen Shrimp After Radioactiv...,https://www.nytimes.com/2025/08/20/business/ra...
4,2025-08-21-16-46-21 +0000,startribune,Live: New foods take center stage on sunny fir...,https://www.startribune.com/mn-state-fair-food...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2344/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
4,trump,54
27,new,26
233,ukraine,18
227,russia,14
220,israel,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
290,2025-08-20-20-36-51 +0000,nypost,Kremlin’s new demands threaten to derail peace...,https://nypost.com/2025/08/20/us-news/kremlins...,133
26,2025-08-21-16-15-09 +0000,startribune,Appeals court throws out massive civil fraud p...,https://www.startribune.com/appeals-court-thro...,115
62,2025-08-21-15-01-46 +0000,nypost,Trump’s massive $500M civil fraud fine in AG T...,https://nypost.com/2025/08/21/us-news/trumps-m...,108
11,2025-08-21-16-38-00 +0000,wsj,N.Y. Appeals Court Throws Out $500 Million Civ...,https://www.wsj.com/us-news/law/ny-appeals-cou...,106
108,2025-08-21-13-22-32 +0000,nypost,Russian missiles hit US-owned factory in Ukrai...,https://nypost.com/2025/08/21/world-news/russi...,104


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
290,133,2025-08-20-20-36-51 +0000,nypost,Kremlin’s new demands threaten to derail peace...,https://nypost.com/2025/08/20/us-news/kremlins...
26,61,2025-08-21-16-15-09 +0000,startribune,Appeals court throws out massive civil fraud p...,https://www.startribune.com/appeals-court-thro...
163,50,2025-08-21-09-39-40 +0000,bbc,Palestinians flee Gaza City districts as Israe...,https://www.bbc.com/news/articles/clyr7l0z9edo...
121,48,2025-08-21-12-30-03 +0000,nypost,US keeping lofty EU car tariffs until levies o...,https://nypost.com/2025/08/21/us-news/us-keepi...
250,44,2025-08-20-22-40-00 +0000,wsj,Fed minutes from the July meeting reveal broad...,https://www.wsj.com/economy/central-banking/fe...
193,42,2025-08-21-05-56-59 +0000,nypost,Screaming man who tried to run from federal of...,https://nypost.com/2025/08/21/us-news/dc-migra...
49,41,2025-08-21-15-24-02 +0000,nypost,Husband of missing California mom of four char...,https://nypost.com/2025/08/21/us-news/nikki-sa...
237,40,2025-08-21-00-19-00 +0000,wsj,President Trump has ordered the Pentagon to se...,https://www.wsj.com/politics/national-security...
87,39,2025-08-21-14-13-25 +0000,cbc,"Judge Frank Caprio, known online as the 'nices...",https://www.cbc.ca/news/world/judge-frank-capr...
281,36,2025-08-20-20-57-46 +0000,nypost,"JD Vance, Stephen Miller mock ‘old, white’ pro...",https://nypost.com/2025/08/20/us-news/jd-vance...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
